## **WALKING 20 MINUTES**

## Loading in Imports and Datasets

**Imports:**

In [1]:
import requests
from urllib.request import urlopen
import json
from shapely.geometry import shape, Polygon
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
plt.style.use('default')
import geopandas as gpd
import geodatasets
import ast
from shapely import wkt
from pyproj import Geod

**Library Branch Info for each year (2011-2023):**

In [2]:
BranchInfo = pd.read_csv('../../../notebooks/Dani/CSV Compile for Library Info and Nbhs/yearlyBranchInfo.csv')

In [3]:
#Checking
BranchInfo

,YEAR,BRANCH,MONTH,CIRCULATION,COMPUTER_SESSIONS,VISITORS
0,2011,Albany Park,78,120059.0,21966.0,117425.0
1,2011,Altgeld,78,9611.0,31782.0,58657.0
2,2011,Archer Heights,78,101951.0,25321.0,124653.0
3,2011,Austin,78,25527.0,33207.0,102306.0
4,2011,Austin-Irving,78,165634.0,35093.0,145852.0
...,...,...,...,...,...,...
1007,2023,West Pullman,10,3325.0,4629.0,23966.0
1008,2023,West Town,10,18885.0,4329.0,29180.0
1009,2023,"Whitney M. Young, Jr.",10,3068.0,6402.0,14232.0
1010,2023,Woodson Regional,10,8480.0,12663.0,24108.0


**Average Travel Time to Each Library (within defined Neighborhoods):**

In [9]:
avgLibTravelTimes = pd.read_csv('../../../data/final_data/averageWalkingTimeNeighborhoods.csv')

In [10]:
#Checking, removing unneccessary column, renaming columns to match BranchInfo for later when merging
avgLibTravelTime = avgLibTravelTimes.drop(columns=['Unnamed: 0']).rename(columns= {"Branch":"BRANCH", "Average Walking Time (Minutes)":"AVG WALKING TIME (MIN)"})
#Replacing Library name to match with BranchInfo
avgLibTravelTime['BRANCH'] = avgLibTravelTime['BRANCH'].replace(['Woodson Regional Library'], 'Woodson Regional')
avgLibTravelTime

,BRANCH,AVG WALKING TIME (MIN)
0,Albany Park,13.228101
1,Altgeld,18.833803
2,Archer Heights,15.424347
3,Austin,14.815993
4,Austin-Irving,13.433197
...,...,...
76,West Pullman,19.417139
77,West Town,11.655752
78,"Whitney M. Young, Jr.",15.007914
79,Woodson Regional,16.768074


## Cleaning DataFrame and Adding in Library Averages (CSPV, CPV)

In [11]:
#Index is by Year
#Dropping Month column, renaming Computer Sessions column
yearlyBranchInfo = BranchInfo.groupby(['YEAR','BRANCH']).sum().drop(columns=['MONTH']).rename(columns= {'COMPUTER_SESSIONS':'COMPUTER SESSIONS'})

#Calculating Computer Sessions Per Visitor
yearlyBranchInfo['AVG COMPUTER SESSIONS PER VISITOR'] = (yearlyBranchInfo['COMPUTER SESSIONS']/ yearlyBranchInfo['VISITORS'])

#Calculating Circulation Per Visitor
yearlyBranchInfo['AVG CIRCULATION PER VISITOR'] = (yearlyBranchInfo['CIRCULATION']/ yearlyBranchInfo['VISITORS'])
yearlyBranchInfo


CIRCULATION  COMPUTER SESSIONS  VISITORS  \
YEAR BRANCH                                                            
2011 Albany Park               120059.0            21966.0  117425.0   
     Altgeld                     9611.0            31782.0   58657.0   
     Archer Heights            101951.0            25321.0  124653.0   
     Austin                     25527.0            33207.0  102306.0   
     Austin-Irving             165634.0            35093.0  145852.0   
...                                 ...                ...       ...   
2023 West Pullman                3325.0             4629.0   23966.0   
     West Town                  18885.0             4329.0   29180.0   
     Whitney M. Young, Jr.       3068.0             6402.0   14232.0   
     Woodson Regional            8480.0            12663.0   24108.0   
     Wrightwood-Ashburn          3213.0             2701.0   24157.0   

                            AVG COMPUTER SESSIONS PER VISITOR  \
YEAR BRANCH                                                     
2011 Albany Park                                     0.187064   
     Altgeld                                         0.541828   
     Archer Heights                                  0.203132   
     Austin                                          0.324585   
     Austin-Irving                                   0.240607   
...                                                       ...   
2023 West Pullman                                    0.193149   
     West Town                                       0.148355   
     Whitney M. Young, Jr.                           0.449831   
     Woodson Regional                                0.525261   
     Wrightwood-Ashburn                              0.111810   

                            AVG CIRCULATION PER VISITOR  
YEAR BRANCH                                              
2011 Albany Park                               1.022431  
     Altgeld                                   0.163851  
     Archer Heights                            0.817878  
     Austin                                    0.249516  
     Austin-Irving                             1.135631  
...                                                 ...  
2023 West Pullman                              0.138738  
     West Town                                 0.647190  
     Whitney M. Young, Jr.                     0.215571  
     Woodson Regional                          0.351750  
     Wrightwood-Ashburn                        0.133005  

[1012 rows x 5 columns]

### 2019

In [12]:
#Extracting only info for each branch for 2019
year2019changes = yearlyBranchInfo.loc[[2019]]

#Renaming columns to specify for only 2019
year2019 = year2019changes.reset_index().drop(columns=['YEAR']).rename(columns= {'CIRCULATION':'CIRCULATION_2019', 
                                                                                 'COMPUTER SESSIONS':'COMPUTER SESSIONS_2019',
                                                                                 'VISITORS':'VISITORS_2019',
                                                                                 'AVG COMPUTER SESSIONS PER VISITOR':'AVG COMPUTER SESSIONS PER VISITOR_2019',
                                                                                 'AVG CIRCULATION PER VISITOR':'AVG CIRCULATION PER VISITOR_2019'})
year2019

,BRANCH,CIRCULATION_2019,COMPUTER SESSIONS_2019,VISITORS_2019,AVG COMPUTER SESSIONS PER VISITOR_2019,AVG CIRCULATION PER VISITOR_2019
0,Albany Park,93009.0,23279.0,130255.0,0.178719,0.714053
1,Altgeld,4014.0,5226.0,28196.0,0.185345,0.142361
2,Archer Heights,58025.0,17466.0,86593.0,0.201702,0.670089
3,Austin,11328.0,16288.0,77480.0,0.210222,0.146205
4,Austin-Irving,119100.0,16115.0,109080.0,0.147736,1.091859
...,...,...,...,...,...,...
73,West Pullman,17823.0,26144.0,87708.0,0.298080,0.203208
74,West Town,59889.0,15214.0,102826.0,0.147959,0.582431
75,"Whitney M. Young, Jr.",21973.0,25752.0,82023.0,0.313961,0.267888
76,Woodson Regional,59933.0,86153.0,214173.0,0.402259,0.279835


### 2021

In [13]:
#Extracting only info for each branch for 2021
year2021changes = yearlyBranchInfo.loc[[2021]]

#Renaming columns to specify for only 2021
year2021 = year2021changes.reset_index().drop(columns=['YEAR']).rename(columns= {'CIRCULATION':'CIRCULATION_2021', 
                                                                                 'COMPUTER SESSIONS':'COMPUTER SESSIONS_2021',
                                                                                 'VISITORS':'VISITORS_2021',
                                                                                 'AVG COMPUTER SESSIONS PER VISITOR':'AVG COMPUTER SESSIONS PER VISITOR_2021',
                                                                                 'AVG CIRCULATION PER VISITOR':'AVG CIRCULATION PER VISITOR_2021'})
year2021

,BRANCH,CIRCULATION_2021,COMPUTER SESSIONS_2021,VISITORS_2021,AVG COMPUTER SESSIONS PER VISITOR_2021,AVG CIRCULATION PER VISITOR_2021
0,Albany Park,61634.0,8838.0,62840.0,0.140643,0.980808
1,Altgeld,1934.0,6719.0,10760.0,0.624442,0.179740
2,Archer Heights,26320.0,7223.0,34665.0,0.208366,0.759267
3,Austin,7108.0,8554.0,35321.0,0.242179,0.201240
4,Austin-Irving,62934.0,5423.0,48720.0,0.111310,1.291749
...,...,...,...,...,...,...
75,West Pullman,7361.0,9520.0,15932.0,0.597540,0.462026
76,West Town,45464.0,7276.0,48278.0,0.150710,0.941713
77,"Whitney M. Young, Jr.",8142.0,13462.0,30166.0,0.446264,0.269907
78,Woodson Regional,23328.0,28566.0,55987.0,0.510226,0.416668


In [14]:
#Merging to create new column that shows average walking time for each neighborhood
year2021_walk = year2021.merge(avgLibTravelTime, how='outer', on='BRANCH')
year2021_walk

,BRANCH,CIRCULATION_2021,COMPUTER SESSIONS_2021,VISITORS_2021,AVG COMPUTER SESSIONS PER VISITOR_2021,AVG CIRCULATION PER VISITOR_2021,AVG WALKING TIME (MIN)
0,Albany Park,61634.0,8838.0,62840.0,0.140643,0.980808,13.228101
1,Altgeld,1934.0,6719.0,10760.0,0.624442,0.179740,18.833803
2,Archer Heights,26320.0,7223.0,34665.0,0.208366,0.759267,15.424347
3,Austin,7108.0,8554.0,35321.0,0.242179,0.201240,14.815993
4,Austin-Irving,62934.0,5423.0,48720.0,0.111310,1.291749,13.433197
...,...,...,...,...,...,...,...
76,West Town,45464.0,7276.0,48278.0,0.150710,0.941713,11.655752
77,"Whitney M. Young, Jr.",8142.0,13462.0,30166.0,0.446264,0.269907,15.007914
78,Woodson Regional,23328.0,28566.0,55987.0,0.510226,0.416668,16.768074
79,Wrightwood-Ashburn,6840.0,4584.0,12274.0,0.373472,0.557276,15.063057


In [15]:
#Merging 2019 and 2021 Information into one dataframe
libInfo2019_2021 = year2019.merge(year2021_walk, how='outer', on='BRANCH')
libInfo2019_2021

,BRANCH,CIRCULATION_2019,COMPUTER SESSIONS_2019,VISITORS_2019,AVG COMPUTER SESSIONS PER VISITOR_2019,AVG CIRCULATION PER VISITOR_2019,CIRCULATION_2021,COMPUTER SESSIONS_2021,VISITORS_2021,AVG COMPUTER SESSIONS PER VISITOR_2021,AVG CIRCULATION PER VISITOR_2021,AVG WALKING TIME (MIN)
0,Albany Park,93009.0,23279.0,130255.0,0.178719,0.714053,61634.0,8838.0,62840.0,0.140643,0.980808,13.228101
1,Altgeld,4014.0,5226.0,28196.0,0.185345,0.142361,1934.0,6719.0,10760.0,0.624442,0.179740,18.833803
2,Archer Heights,58025.0,17466.0,86593.0,0.201702,0.670089,26320.0,7223.0,34665.0,0.208366,0.759267,15.424347
3,Austin,11328.0,16288.0,77480.0,0.210222,0.146205,7108.0,8554.0,35321.0,0.242179,0.201240,14.815993
4,Austin-Irving,119100.0,16115.0,109080.0,0.147736,1.091859,62934.0,5423.0,48720.0,0.111310,1.291749,13.433197
...,...,...,...,...,...,...,...,...,...,...,...,...
76,Woodson Regional,59933.0,86153.0,214173.0,0.402259,0.279835,23328.0,28566.0,55987.0,0.510226,0.416668,16.768074
77,Wrightwood-Ashburn,17607.0,12697.0,47353.0,0.268135,0.371824,6840.0,4584.0,12274.0,0.373472,0.557276,15.063057
78,Douglass,NaN,NaN,NaN,NaN,NaN,4852.0,7210.0,8045.0,0.896209,0.603108,14.027758
79,Water Works,NaN,NaN,NaN,NaN,NaN,8516.0,306.0,10594.0,0.028884,0.803851,11.544491


In [16]:
libInfo2019_2021.to_csv('../../../data/final_data/libInfo2019_2021.csv')